In [1]:
import os


In [2]:
pwd

'd:\\DataScience\\End-to-End-Chicken-disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DataScience\\End-to-End-Chicken-disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngestionConfig:
    root_dir : Path 
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [7]:
from src.cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(self,config_path = CONFIG_FILE_PATH,
                 params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_URL = config.source_URL,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from cnnClassifier.utils.common import get_size
from cnnClassifier import logger


In [13]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file} with size {get_size(self.config.local_data_file)}")
    def extract_zip_file(self):
            unzip_path  =self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
                zip_ref.extractall(unzip_path)

In [14]:
try:
    obj = ConfigurationManager()
    data_ingestion_config = obj.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-27 20:10:56,074]: INFO: common: YAML file config\config.yaml read successfully.
[2025-07-27 20:10:56,077]: INFO: common: YAML file params.yaml read successfully.
[2025-07-27 20:10:56,077]: INFO: common: Created directory: artifacts
[2025-07-27 20:10:56,082]: INFO: common: Created directory: artifacts/data_ingestion
[2025-07-27 20:10:56,082]: INFO: 1975770095: File already exists at artifacts\data_ingestion\data.zip with size 25182 KB
